In [ ]:
import json
import os
import numpy as np
import time
from datetime import datetime, timedelta
from collections import defaultdict

print(f"WinRank46v04 - INCREMENTAL NPA WINDOW RANKING - {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

def find_latest_npa_file():
    """Find the most recent NPA positions file"""
    print("Finding latest NPA position file...")
    
    # Check multiple possible locations
    possible_paths = ['../outputs', 'outputs', '../../outputs']
    outputs_path = None
    
    for path in possible_paths:
        if os.path.exists(path):
            outputs_path = path
            print(f"✅ Found outputs folder: {path}")
            break
    
    if not outputs_path:
        print("❌ outputs folder not found in any location")
        print("   Checked: ../outputs, outputs, ../../outputs")
        return None
    
    npa_files = []
    for filename in os.listdir(outputs_path):
        if filename.startswith('npa_positions_') and filename.endswith('.json'):
            filepath = os.path.join(outputs_path, filename)
            mtime = os.path.getmtime(filepath)
            size = os.path.getsize(filepath)
            npa_files.append((filename, mtime, size))
    
    if not npa_files:
        print("❌ No NPA position files found")
        return None
    
    # Sort by modification time (newest first)
    npa_files.sort(key=lambda x: x[1], reverse=True)
    latest_file = npa_files[0][0]
    
    print(f"✅ Using latest NPA file: {latest_file} ({npa_files[0][2]:,} bytes)")
    return os.path.join(outputs_path, latest_file)

def incremental_window_analysis(family_npa_data, window_size=60, step_size=20, min_npas=5, save_interval=60):
    """Incremental sliding window analysis with frequent saves"""
    
    print(f"🔄 Starting incremental analysis...")
    print(f"   Window size: {window_size} SNPs, Step: {step_size} SNPs, Min NPAs: {min_npas}")
    
    all_windows = []
    families_to_process = list(family_npa_data.items())
    families_completed = 0
    
    print(f"📊 Processing {len(families_to_process)} families...")
    
    analysis_start = time.time()
    last_save = analysis_start
    last_progress = analysis_start
    
    for family_idx, (child_id, fam_data) in enumerate(families_to_process):
        current_time = time.time()
        
        # Progress message every 10 seconds
        if current_time - last_progress >= 10:
            elapsed = (current_time - analysis_start) / 60
            progress_pct = ((family_idx + 1) / len(families_to_process)) * 100
            print(f"⏱️  {elapsed:.1f}min | Family {family_idx+1}/{len(families_to_process)} ({progress_pct:.1f}%) | Windows found: {len(all_windows)}")
            last_progress = current_time
        
        npa_positions = fam_data.get('npa_positions', [])
        
        if len(npa_positions) < min_npas:
            families_completed += 1
            continue
        
        # Show families with significant data
        if len(npa_positions) >= 10:
            print(f"🔍 Analyzing {child_id}: {len(npa_positions)} NPAs")
        
        sorted_positions = sorted(npa_positions)
        min_pos = min(sorted_positions)
        max_pos = max(sorted_positions)
        
        if max_pos == min_pos:
            families_completed += 1
            continue
        
        # Sliding windows for this family
        snp_spacing = 1000  # bp per SNP estimate
        window_bp_size = window_size * snp_spacing
        step_bp_size = step_size * snp_spacing
        
        current_start = min_pos
        family_windows = 0
        
        while current_start <= max_pos:
            window_end = current_start + window_bp_size
            
            npas_in_window = [pos for pos in sorted_positions 
                             if current_start <= pos < window_end]
            
            if len(npas_in_window) >= min_npas:
                all_windows.append({
                    'child_id': child_id,
                    'window_start': current_start,
                    'window_end': window_end,
                    'window_center': (current_start + window_end) // 2,
                    'npa_count': len(npas_in_window),
                    'window_density': len(npas_in_window) / window_size
                })
                family_windows += 1
            
            current_start += step_bp_size
        
        families_completed += 1
        
        if family_windows > 0:
            print(f"🔥 {child_id}: {family_windows} significant windows found!")
        
        # Save partial results every minute
        current_time = time.time()
        if current_time - last_save >= save_interval:
            print(f"💾 Saving partial results...")
            save_partial_results(all_windows, families_completed, len(families_to_process), 
                               (current_time - analysis_start) / 60)
            last_save = current_time
    
    print(f"✅ Analysis complete: {families_completed} families processed")
    return all_windows, families_completed

def save_partial_results(windows, families_done, total_families, elapsed_minutes):
    """Save partial results to disk"""
    
    if not windows:
        print(f"📊 {elapsed_minutes:.1f}min | Families: {families_done}/{total_families} | No significant windows yet")
        return
    
    # Rank current windows
    ranked_windows = sorted(windows, key=lambda x: x['npa_count'], reverse=True)
    
    # Top 20 for quick display
    top_20 = ranked_windows[:20]
    
    # Statistics
    npa_counts = [w['npa_count'] for w in ranked_windows]
    families_with_clusters = len(set(w['child_id'] for w in ranked_windows))
    
    # Save to disk
    timestamp = datetime.now().strftime('%H%M%S')
    
    # Use same outputs path logic
    possible_paths = ['../outputs', 'outputs', '../../outputs']
    outputs_path = None
    for path in possible_paths:
        if os.path.exists(path):
            outputs_path = path
            break
    
    if not outputs_path:
        # Create outputs in parent if it doesn't exist
        outputs_path = '../outputs'
        os.makedirs(outputs_path, exist_ok=True)
        print(f"📁 Created outputs folder: {outputs_path}")
    
    partial_file = os.path.join(outputs_path, f'partial_ranking_{timestamp}.json')
    latest_file = os.path.join(outputs_path, 'partial_ranking_LATEST.json')
    
    save_data = {
        'analysis_time': datetime.now().isoformat(),
        'status': 'PARTIAL',
        'progress': {
            'families_completed': families_done,
            'total_families': total_families,
            'percent_complete': (families_done / total_families) * 100 if total_families > 0 else 0,
            'elapsed_minutes': elapsed_minutes
        },
        'current_stats': {
            'total_windows': len(ranked_windows),
            'families_with_clusters': families_with_clusters,
            'max_npas': max(npa_counts) if npa_counts else 0,
            'mean_npas': np.mean(npa_counts) if npa_counts else 0,
            'top_20_windows': top_20
        }
    }
    
    with open(partial_file, 'w') as f:
        json.dump(save_data, f, indent=2)
    
    # Always update LATEST file
    with open(latest_file, 'w') as f:
        json.dump(save_data, f, indent=2)
    
    # Screen output
    progress_pct = (families_done / total_families) * 100 if total_families > 0 else 0
    max_npas = max(npa_counts) if npa_counts else 0
    print(f"📊 {elapsed_minutes:.1f}min | Families: {families_done}/{total_families} ({progress_pct:.1f}%) | Windows: {len(ranked_windows)} | Max NPAs: {max_npas}")
    print(f"💾 Saved: {partial_file}")
    
    # Show top 3 current windows
    if len(top_20) > 0:
        print(f"🏆 Top 3 windows:")
        for i, w in enumerate(top_20[:3], 1):
            print(f"   {i}. {w['child_id']}: {w['npa_count']} NPAs at {w['window_center']:,}")

def main_analysis_run():
    """Single analysis run"""
    
    print(f"🚀 Starting analysis run...")
    
    # Find and load latest NPA data
    latest_file = find_latest_npa_file()
    if not latest_file:
        print("❌ No NPA data files available")
        return None
    
    print(f"📖 Loading NPA data from {latest_file}...")
    try:
        with open(latest_file, 'r') as f:
            npa_data = json.load(f)
        print(f"✅ NPA data loaded successfully")
    except Exception as e:
        print(f"❌ Error reading NPA file: {e}")
        return None
    
    family_npa_data = npa_data.get('family_npa_data', {})
    
    if not family_npa_data:
        print(f"❌ No family NPA data found in file")
        return None
    
    print(f"📊 Found NPA data for {len(family_npa_data)} families")
    
    # Show source analysis progress
    chr3_percent = 0
    if 'progress' in npa_data:
        progress = npa_data['progress']
        elapsed = progress.get('elapsed_minutes', 0)
        position = progress.get('snp_position', 0)
        chr3_percent = progress.get('chr3_percent', 0)
        print(f"📈 Source analysis: {elapsed:.1f}min elapsed, {chr3_percent:.1f}% of chr3 complete ({position:,} bp)")
    
    # Run incremental analysis
    print(f"🔄 Starting window analysis...")
    all_windows, families_completed = incremental_window_analysis(family_npa_data)
    
    # Final save with HUMAN-READABLE SUMMARY
    if all_windows:
        ranked_windows = sorted(all_windows, key=lambda x: x['npa_count'], reverse=True)
        
        print(f"\n🏁 ANALYSIS COMPLETE - FOUND {len(ranked_windows)} SIGNIFICANT WINDOWS")
        print(f"=" * 60)
        print(f"TOP 10 NPA CLUSTERS:")
        print(f"Rank  Family      NPAs  Position      Density")
        print(f"----  ----------  ----  -----------   -------")
        
        for i, w in enumerate(ranked_windows[:10], 1):
            child_id = w['child_id']
            npa_count = w['npa_count']
            center = w['window_center']
            density = w['window_density']
            print(f"{i:2d}    {child_id:<10}  {npa_count:2d}    {center:>10,}   {density:.3f}")
        
        # Summary stats
        npa_counts = [w['npa_count'] for w in ranked_windows]
        families_with_clusters = len(set(w['child_id'] for w in ranked_windows))
        
        print(f"=" * 60)
        print(f"SUMMARY STATISTICS:")
        print(f"• Total significant windows: {len(ranked_windows)}")
        print(f"• Families with clusters: {families_with_clusters}/{families_completed}")
        print(f"• Strongest cluster: {max(npa_counts)} NPAs")
        print(f"• Average cluster: {np.mean(npa_counts):.1f} NPAs")
        print(f"• Source data: {chr3_percent:.1f}% of chr3 analyzed")
        
        final_data = {
            'analysis_time': datetime.now().isoformat(),
            'status': 'COMPLETE',
            'source_file': latest_file,
            'total_windows': len(ranked_windows),
            'families_analyzed': families_completed,
            'top_50_windows': ranked_windows[:50]
        }
        
        timestamp = datetime.now().strftime('%H%M%S')
        final_file = f'outputs/window_ranking_COMPLETE_{timestamp}.json'
        
        with open(final_file, 'w') as f:
            json.dump(final_data, f, indent=2)
        
        print(f"💾 Complete analysis saved: {final_file}")
        return len(ranked_windows)
    else:
        print(f"❌ No significant windows found")
    
    return 0

# Main continuous loop
print(f"=" * 60)
print(f"🔄 Starting continuous incremental monitoring")

run_count = 0
while True:
    run_count += 1
    print(f"\n=== RUN #{run_count} at {datetime.now().strftime('%H:%M:%S')} ===")
    
    try:
        windows_found = main_analysis_run()
        
        if windows_found is not None:
            print(f"✅ Run #{run_count} complete - found {windows_found} total windows")
        else:
            print(f"⚠️  Run #{run_count} complete - no data available")
        
    except Exception as e:
        print(f"❌ Error in run #{run_count}: {e}")
        import traceback
        traceback.print_exc()
    
    print(f"⏱️  Waiting 5 minutes...")
    print(f"⏰ Next run: {(datetime.now() + timedelta(minutes=5)).strftime('%H:%M:%S')}")
    time.sleep(300)  # 5 minutes

WinRank46v04 - INCREMENTAL NPA WINDOW RANKING - 2025-05-29 14:57:57
🔄 Starting continuous incremental monitoring

=== RUN #1 at 14:57:57 ===
🚀 Starting analysis run...
Finding latest NPA position file...
✅ Found outputs folder: ../outputs
✅ Using latest NPA file: npa_positions_CHR1_COMPLETE_v13.json (694,337 bytes)
📖 Loading NPA data from ../outputs\npa_positions_CHR1_COMPLETE_v13.json...
✅ NPA data loaded successfully
📊 Found NPA data for 602 families
🔄 Starting window analysis...
🔄 Starting incremental analysis...
   Window size: 60 SNPs, Step: 20 SNPs, Min NPAs: 5
📊 Processing 602 families...
🔍 Analyzing HG01881: 85 NPAs
🔍 Analyzing HG01888: 140 NPAs
🔥 HG01888: 11 significant windows found!
🔍 Analyzing HG01884: 76 NPAs
🔍 Analyzing HG01887: 58 NPAs
🔍 Analyzing HG01891: 101 NPAs
🔥 HG01891: 7 significant windows found!
🔍 Analyzing HG01895: 135 NPAs
🔥 HG01895: 15 significant windows found!
🔍 Analyzing HG01897: 75 NPAs
🔥 HG01897: 4 significant windows found!
🔍 Analyzing HG01916: 104 NPAs

Traceback (most recent call last):
  File "C:\Users\mremp\AppData\Local\Temp\ipykernel_5072\3378563032.py", line 311, in <module>
    windows_found = main_analysis_run()
                    ^^^^^^^^^^^^^^^^^^^
  File "C:\Users\mremp\AppData\Local\Temp\ipykernel_5072\3378563032.py", line 291, in main_analysis_run
    with open(final_file, 'w') as f:
         ^^^^^^^^^^^^^^^^^^^^^
  File "C:\ProgramData\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py", line 324, in _modified_open
    return io_open(file, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
FileNotFoundError: [Errno 2] No such file or directory: 'outputs/window_ranking_COMPLETE_145809.json'
